In [20]:
import monai
import torch

image_dir_no_patho = ["./data/ATLAS/splits_healthy_54_sample/Atlas_healthy.csv"]
image_dir_patho_reference = ["./data/ATLAS/splits_reference_test_unhealthy_54_samples/atlas_test_png.csv"]
image_dir_patho_reference_same_atlas = ["./data/ATLAS/splits_over_1_stratified_all_splits/atlas_test_png.csv"]

import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from torch.utils.data import DataLoader
import sys
import os

os.chdir('/home/malek/mock/autoDDPM')
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))


# Get the parent directory
# Add the parent directory to the system path
sys.path.append(dir_path)
print(dir_path)
from data.loaders.ixi_loader import mask_preprocessing_loader, AtlasLoader, IXILoaderHarmonize, AtlasLoaderHarmonize

from PIL import Image
import numpy as np


device = "cuda" if torch.cuda.is_available() else "cpu"

dataset_no_patho = AtlasLoader(
            image_dir_no_patho,
            target_size=(128, 128),
            test=False,
        )

loader_no_patho = DataLoader(
    dataset_no_patho, batch_size=54, shuffle=False, drop_last=False, pin_memory=True
)

dataset_patho_referemce = AtlasLoader(
            image_dir_patho_reference,
            target_size=(128, 128),
            test=False,
        )

loader_patho_reference = DataLoader(
    dataset_patho_referemce, batch_size=54, shuffle=False, drop_last=False, pin_memory=True
)

dataset_patho_referemce_same_atlas = AtlasLoader(
            image_dir_patho_reference_same_atlas,
            target_size=(128, 128),
            test=False,
        )

loader_patho_reference_same_atlas = DataLoader(
    dataset_patho_referemce_same_atlas, batch_size=54, shuffle=False, drop_last=False, pin_memory=True
)




/home/malek/mock


In [21]:
from generative.metrics import FIDMetric, MMDMetric, MultiScaleSSIMMetric, SSIMMetric

radnet = torch.hub.load("Warvito/radimagenet-models:main", model="radimagenet_resnet50",verbose=True)

radnet.to(device)
radnet.eval()

Using cache found in /home/malek/.cache/torch/hub/Warvito_radimagenet-models_main


ResNet50(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1.001e-05, momentum=0.01, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(64, eps=1.001e-05, momentum=0.99, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1.001e-05, momentum=0.99, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn3): BatchNorm2d(256, eps=1.001e-05, momentum=0.99, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm

In [22]:
def subtract_mean(x: torch.Tensor) -> torch.Tensor:
    mean = [0.406, 0.456, 0.485]
    x[:, 0, :, :] -= mean[0]
    x[:, 1, :, :] -= mean[1]
    x[:, 2, :, :] -= mean[2]
    return x


def spatial_average(x: torch.Tensor, keepdim: bool = True) -> torch.Tensor:
    return x.mean([2, 3], keepdim=keepdim)


def get_features(image):
    # If input has just 1 channel, repeat channel to have 3 channels
    if image.shape[1]:
        image = image.repeat(1, 3, 1, 1)

    # Change order from 'RGB' to 'BGR'
    image = image[:, [2, 1, 0], ...]

    # Subtract mean used during training
    image = subtract_mean(image)

    # Get model outputs
    with torch.no_grad():
        feature_image = radnet.forward(image)
        # flattens the image spatially
        feature_image = spatial_average(feature_image, keepdim=False)

    return feature_image


In [23]:
for (data_1,data_2,data_3) in zip(loader_no_patho,loader_patho_reference,loader_patho_reference_same_atlas):
    images_no_patho = data_1[0].to(device)
    images_patho_reference = data_2[0].to(device)
    images_patho_reference_same_atlas = data_3[0].to(device)
    
    
    print("shape of images_no_patho",images_no_patho.shape)



shape of images_no_patho torch.Size([54, 1, 128, 128])


In [24]:
from ast import List
from dl_utils.fid_score import save_fid_stats, calculate_fid_given_images
device = torch.device('cuda' if (torch.cuda.is_available()) else 'cpu')
print('device:', device)
num_workers = 4

List_of_images = [images_no_patho,images_patho_reference]

fid_value = calculate_fid_given_images(List_of_images,
                                        54,
                                        device,
                                        2048,
                                        num_workers)

#print('FID: ', fid_value)

device: cuda
shape of tensor torch.Size([54, 3, 128, 128])
shape of tensor torch.Size([54, 3, 128, 128])


In [25]:
synth_features = []
real_features = []

real_eval_feats = get_features(images_no_patho)
#print('real_eval_feats shape',real_eval_feats.shape)
real_features.append(real_eval_feats)
#print('real features shape ',len(real_features))

# Get the features for the synthetic data
synth_eval_feats = get_features(images_patho_reference)
synth_features.append(synth_eval_feats)

synth_features = torch.vstack(synth_features)
real_features = torch.vstack(real_features)
#print('shape of real features tensor',real_features.shape)

fid = FIDMetric()
fid_res = fid(synth_features, real_features)

print('FID score Radnet between pathological (reference) and non pathological Atlas subsets')
print(f"FID Score Radnet: {fid_res.item():.2f}")
print(f'FID score InceptionNet: {fid_value:.2f}')

FID score Radnet between pathological (reference) and non pathological Atlas subsets
FID Score Radnet: 6.30
FID score InceptionNet: 40.84


In [26]:
List_of_images = [images_no_patho,images_patho_reference_same_atlas]

fid_value = calculate_fid_given_images(List_of_images,
                                        54,
                                        device,
                                        2048,
                                        num_workers)

#print('FID: ', fid_value)

shape of tensor torch.Size([54, 3, 128, 128])
shape of tensor torch.Size([54, 3, 128, 128])


In [57]:
def compute_fid_radnet(images_1,images_2):
    synth_features = []
    real_features = []

    real_eval_feats = get_features(images_1)
    real_features.append(real_eval_feats)

    # Get the features for the synthetic data
    synth_eval_feats = get_features(images_2)
    synth_features.append(synth_eval_feats)

    synth_features = torch.vstack(synth_features)
    real_features = torch.vstack(real_features)

    fid = FIDMetric()
    fid_res = fid(synth_features, real_features)
    fid_res= round(fid_res.item(), 2)

    return fid_res


In [58]:
fid_res = compute_fid_radnet(images_no_patho,images_patho_reference_same_atlas)
print('FID score between pathological (reference same atlas split) and non pathological Atlas subsets')
print(f"FID Score: {fid_res}")

print(f"FID Score InceptionNet: {fid_value:.2f}")

FID score between pathological (reference same atlas split) and non pathological Atlas subsets
FID Score: 7.9
FID Score InceptionNet: 35.39


In [9]:
from torch.utils.data import DataLoader, Dataset, RandomSampler


In [46]:
from torch.utils.data import DataLoader, Dataset, RandomSampler

torch.manual_seed(42)
# Create a list to store the loaders
loaders_patho_reference_same_atlas = []

# Loop 10 times to create multiple instances of the loaders
for _ in range(10):
    # Create a new instance of AtlasLoader
    dataset_patho_reference_same_atlas = AtlasLoader(
        image_dir_patho_reference_same_atlas,
        target_size=(128, 128),
        test=False,
    )
    sampler = RandomSampler(dataset_patho_reference_same_atlas, replacement=True)

    # Create a new instance of DataLoader
    loader_patho_reference_same_atlas = DataLoader(
        dataset_patho_reference_same_atlas, batch_size=54, shuffle=False, drop_last=False, pin_memory=True, sampler=sampler
    )

    # Append the new loaders to the list
    loaders_patho_reference_same_atlas.append(loader_patho_reference_same_atlas)



In [55]:
from numpy import std

sum_ = 0
fids = []
for data in loaders_patho_reference_same_atlas:
    for images in data:
        images_patho_reference_same_atlas = images[0].to(device)
        #print("shape of images_patho_reference_same_atlas",images_patho_reference_same_atlas.shape)
        fid = compute_fid_radnet(images_no_patho,images_patho_reference_same_atlas)
        sum_ += fid
        fids.append(fid)

mean = sum_/10
print(f"Mean FID Score: {mean:.2f}")
std_ = std(fids)
print(f"Standard Deviation: {std_:.2f}")


FID score between pathological (reference same atlas split) and non pathological Atlas subsets
FID Score: 9.058
FID score between pathological (reference same atlas split) and non pathological Atlas subsets
FID Score: 8.599
FID score between pathological (reference same atlas split) and non pathological Atlas subsets
FID Score: 7.804
FID score between pathological (reference same atlas split) and non pathological Atlas subsets
FID Score: 8.642
FID score between pathological (reference same atlas split) and non pathological Atlas subsets
FID Score: 8.292
FID score between pathological (reference same atlas split) and non pathological Atlas subsets
FID Score: 7.91
FID score between pathological (reference same atlas split) and non pathological Atlas subsets
FID Score: 8.467
FID score between pathological (reference same atlas split) and non pathological Atlas subsets
FID Score: 8.448
FID score between pathological (reference same atlas split) and non pathological Atlas subsets
FID Score:

In [65]:
import torch

# Assume input is a tensor of shape [b, c, 128, 18]
b, c = 10, 3
input_tensor = torch.rand(b, c, 128, 18)

# Sample with replacement from the tensor 10 times
for _ in range(10):
    # Generate random indices
    indices = torch.randint(b, (b,))
    print(indices)

    # Use the indices to index into the tensor
    sampled_tensor = input_tensor[indices]

    print(sampled_tensor.shape)  # Prints: torch.Size([b, c, 128, 18])


tensor([5, 9, 1, 9, 1, 7, 7, 8, 9, 4])
torch.Size([10, 3, 128, 18])
tensor([8, 3, 1, 5, 7, 9, 5, 5, 5, 1])
torch.Size([10, 3, 128, 18])
tensor([3, 1, 1, 6, 1, 8, 4, 2, 0, 7])
torch.Size([10, 3, 128, 18])
tensor([2, 9, 9, 0, 9, 7, 9, 2, 7, 6])
torch.Size([10, 3, 128, 18])
tensor([0, 2, 2, 7, 6, 9, 4, 5, 6, 6])
torch.Size([10, 3, 128, 18])
tensor([1, 0, 9, 7, 8, 6, 0, 1, 6, 9])
torch.Size([10, 3, 128, 18])
tensor([7, 1, 4, 4, 3, 0, 3, 9, 5, 6])
torch.Size([10, 3, 128, 18])
tensor([7, 1, 1, 0, 6, 1, 4, 7, 1, 0])
torch.Size([10, 3, 128, 18])
tensor([4, 3, 0, 5, 1, 0, 2, 5, 7, 9])
torch.Size([10, 3, 128, 18])
tensor([0, 3, 8, 9, 3, 0, 1, 9, 5, 8])
torch.Size([10, 3, 128, 18])
